In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np

# Set device for PyTorch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

def predict(model, features_scaled, window_size, ticker, scaler, stock_data, enhanced_data, num_quarters=12):  # 12 quarters = 3 years
    model.eval()
    predictions = []
    last_window = features_scaled[-window_size:]
    last_price = float(stock_data['Close'].iloc[-1].iloc[0])
    last_features = enhanced_data.iloc[-1]
    
    # Start from today and generate quarterly dates (every 3 months)
    today = datetime.now()
    prediction_dates = [today + timedelta(days=91 * i) for i in range(1, num_quarters + 1)]
    
    with torch.no_grad():
        current_window = torch.FloatTensor(last_window).unsqueeze(0).to(device)
        current_price = last_price
        
        # Initialize market condition factors
        base_volatility = enhanced_data['20d_vol'].iloc[-1]
        market_trend = 0.02  # Assumed average market growth rate
        
        for i in range(len(prediction_dates)):
            # Predict base percentage change
            pct_change = model(current_window).item()
            
            # Add quarterly seasonality and market cycles
            quarter = (i % 4) + 1
            cycle_adjustment = np.sin(2 * np.pi * i / 8) * 0.02  # Market cycles (2-year cycle)
            
            # Increase volatility over time with randomness
            time_factor = (i + 1) / len(prediction_dates)
            volatility = base_volatility * (1 + time_factor)
            random_factor = np.random.normal(0, volatility)
            
            # Combine all factors for final percentage change
            adjusted_pct_change = (
                pct_change +                     # Base model prediction
                market_trend * time_factor +     # Long-term market trend
                cycle_adjustment +               # Market cycles
                random_factor                    # Random volatility
            )
            
            # Calculate compound growth
            predicted_price = current_price * (1 + adjusted_pct_change)
            
            # Add mean reversion for extreme moves
            if abs(predicted_price/last_price - 1) > 0.5:  # If more than 50% move
                reversion_factor = 0.3  # Strength of mean reversion
                predicted_price = predicted_price * (1 - reversion_factor) + last_price * reversion_factor
            
            predictions.append(predicted_price)
            current_price = predicted_price
            
            # Update volatility based on recent prediction
            base_volatility = base_volatility * 0.9 + abs(adjusted_pct_change) * 0.1
            
            # Create new window
            new_window = current_window.clone()
            new_window[0, :-1, :] = new_window[0, 1:, :]
            
            # Update features with more realistic variations
            new_features = last_features.copy()
            new_features['Close'] = predicted_price
            new_features['Open'] = predicted_price * (1 + np.random.normal(0, 0.005))
            new_features['High'] = predicted_price * (1 + abs(np.random.normal(0, 0.01)))
            new_features['Low'] = predicted_price * (1 - abs(np.random.normal(0, 0.01)))
            new_features['Volume'] = new_features['Volume'] * (1 + np.random.normal(0, 0.15))
            
            # Update technical indicators
            new_features['20d_vol'] = base_volatility
            
            # Scale new features
            scaled_new_features = scaler.transform(new_features.values.reshape(1, -1))[0]
            new_window[0, -1, :] = torch.FloatTensor(scaled_new_features).to(device)
            current_window = new_window

    # Create DataFrame with quarterly predictions
    prediction_df = pd.DataFrame({
        'Date': prediction_dates,
        'Predicted Price': predictions,
        'Quarterly Return': [(p/predictions[i-1] - 1)*100 if i > 0 else (p/last_price - 1)*100 for i, p in enumerate(predictions)],
        'Total Return': [(p/last_price - 1)*100 for p in predictions]
    })
    
    print("\nQuarterly Predictions:")
    print(prediction_df.to_string(float_format=lambda x: '{:.2f}'.format(x)))
    
    # Enhanced visualization
    plt.figure(figsize=(12, 8))
    
    # Plot historical data for context
    plt.plot(stock_data.index[-90:], stock_data['Close'][-90:], label='Historical', color='blue')
    
    # Plot predictions with confidence bands
    plt.plot(prediction_df['Date'], prediction_df['Predicted Price'], 
             label='Predicted', color='green', linestyle='--')
    
    # Add confidence bands
    volatility_band = base_volatility * np.sqrt(np.arange(len(predictions)) + 1)
    upper_band = np.array(predictions) * (1 + volatility_band)
    lower_band = np.array(predictions) * (1 - volatility_band)
    plt.fill_between(prediction_df['Date'], lower_band, upper_band, 
                    alpha=0.2, color='green', label='Confidence Band')
    
    plt.title(f'{ticker} Stock Price Prediction (Quarterly for Next 3 Years)')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return prediction_df


In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import constants
from utils.models.train import train
from utils.models.save import save
from utils.models.load import load
from utils.stocks.download_stock_data import download_stock_data
from utils.stocks.add_long_term_stock_features import add_long_term_stock_features
from utils.stocks.get_breakout_stocks import get_breakout_stocks
from utils.training.long_term_growth import long_term_growth

model_name = "stocks_10x_breakout_model"

# Try to load existing model first
model, scaler, last_data = load(model_name)

if model is None:
    print(f"Training new model.")
    # these tickers are used for training
    # they create the model baseline that we want to beat or match
    reported_breakout_stocks = get_breakout_stocks()

    model, scaler = long_term_growth(reported_breakout_stocks, model_name)
    # Verify we have a NEW model and scaler before proceeding
    if model is None or scaler is None:
        print("No valid model and scaler available. Cannot proceed with predictions.")
        exit()

# Verify we have a PRE_LOADED model and scaler before proceeding
if model is None or scaler is None:
    print("No valid model and scaler available. Cannot proceed with predictions.")
    exit()

# for analysis, these tickers use the model for prediction!
# this is where you set your stock symbols you want to predict on
tickers = ["LCID", "GME", "AMC", "RIOT", "MARA", "PLTR", "NVAX", "EXPE", "UPST", "FUBO", "TSLA", "NIO", "BIDU", "SPCE", "MRNA", "DKNG", "SOFI", "INTC", "LULU", "COIN", "PLUG", "BABA", "FSLY", "WORK", "STPK", "SLGG", "SENS", "CLSK", "VANW", "HOOD"]

# Run the model for each ticker
for ticker in tickers:
    print(f"\nGenerating predictions for {ticker}")
    stock_data = download_stock_data(ticker, 
                                    datetime.now() - timedelta(days=30),
                                    datetime.now())
    enhanced_data = add_long_term_stock_features(stock_data, ticker)
    
    # Define available features (make sure this matches your training features)
    long_term_growth_features = constants.LONG_TERM_STOCK_FEATURES
    features = enhanced_data[long_term_growth_features].values
    if features.size == 0:
        raise ValueError("No valid features extracted")
    features_scaled = scaler.transform(features)
    predictions = predict(model, features_scaled, 90, ticker, scaler, stock_data, enhanced_data, num_quarters=12)